In [ ]:
import pandas as pd
import numpy as np
import os
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import widgets
from IPython.display import display, clear_output
import pickle
import pandas_datareader as pdr
from tqdm.notebook import tqdm as log_progress

### Import Object

In [ ]:
from Forecaster import Forecaster

### Call object API

In [ ]:
f = Forecaster()

### Create a dataframe of x regressors

In [ ]:
def get_regressors():
    pass

### Ingest x regressors

In [ ]:
f.process_xreg_df()

### Make sklearn forecasts

In [ ]:
f.forecast_rf()

### Make R auto.arima forecasts

In [ ]:
f.forecast_auto_arima()

### Make combination model forecast

In [ ]:
f.forecast_average()

### Which is best?

In [ ]:
f.set_best_model()

### Visualize

In [ ]:
f.display_ts_plot()